In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [103]:
import pandas as pd
import pandas_profiling

import numpy as np

import seaborn as sns
sns.set()

from math import sqrt

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

import plotly.plotly as py
import plotly.graph_objs as go

import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("../../data/Sales_Multiseries_training.csv")

In [107]:
def build_model_and_evaluate(X_train, y_train, X_test, y_test):
    
    regressor = Sequential()
    regressor.add(LSTM(units = 3, input_shape = (None, 1)))
    regressor.add(Dense(units = 1))
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    history = regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose=0)
    
    inputs = np.array(X_test)
    inputs = np.reshape(inputs, (inputs.shape[0], inputs.shape[1], 1))
    predicted = regressor.predict(inputs)
    predicted_price = sc.inverse_transform(predicted)
    
    rmse = sqrt(mean_squared_error(y_test, predicted_price))
    print(rmse)

In [108]:
# lets bring in the the week when the sale happened
data["date_"] = pd.to_datetime(arg=data.Date)

In [109]:
def build_data_for_walk_forward_validation(data, storename, date):
    store_data = data[data.Store == storename]
    
    store_data_train = store_data[store_data.date_ < date].iloc[:,2:3].values
    store_data_test = store_data[store_data.date_ >= date].iloc[:,2:3].values
    
    store_data_train_scaled = sc.fit_transform(store_data_train)
    
    # using original values from store data
    X_test_with_prev_data = sc.transform(store_data.iloc[:,2:3].values)
    
    # actual values to compare with
    y_test = store_data_test
    
    
    X_train = []
    y_train = []
    for i in range(7, len(store_data_train_scaled)):
        X_train.append(store_data_train_scaled[i-7:i, 0])
        y_train.append(store_data_train_scaled[i, 0])
        
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
#     # Test data
    X_test = []
    for i in range(len(store_data_train_scaled), len(store_data)):
        X_test.append(X_test_with_prev_data[i-7:i, 0])
            
    build_model_and_evaluate(X_train, y_train, X_test, y_test)

In [110]:
build_data_for_walk_forward_validation(data, "Savannah", "2014-01-01")
build_data_for_walk_forward_validation(data, "Savannah", "2014-02-01")
build_data_for_walk_forward_validation(data, "Savannah", "2014-03-01")
build_data_for_walk_forward_validation(data, "Savannah", "2014-04-01")
build_data_for_walk_forward_validation(data, "Savannah", "2014-05-01")

18680.236157082847
17645.185704640127
17024.80077469559
17652.945795409825
19579.244751112616
